In [10]:
#based on minimal_net cs231n 

In [11]:
import numpy as np
from numpy import exp, array, random, dot

#in this example we will train linear classifier and compare resultes witb nn

D=3
C =2 
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_set_outputs = array([[0, 1, 1, 0]])
#Train a Linear Classifier

#Initialize randomly weights vector(size 2)
np.random.seed(0)
W =np.random.randn(D,C)
b = np.zeros((1,C))

#some hyperparameters

step_size = 1e-0
reg = 1e-3 # regularization strength

# gradient descent loop
num_examples = training_set_inputs.shape[0]
for i in range(200):
    # evaluate class scores, [N x K]
    scores = np.dot(training_set_inputs, W) + b
    # compute the class probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)  # [N x K]

    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),training_set_outputs])
    data_loss = np.sum(corect_logprobs) / num_examples
    reg_loss = 0.5 * reg * np.sum(W * W)
    loss = data_loss + reg_loss
    if i % 10 == 0:
        print("iteration %d: loss %f" % (i, loss))

    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),training_set_outputs] -= 1
    dscores /= num_examples

    # backpropate the gradient to the parameters (W,b)
    dW = np.dot(training_set_inputs.T, dscores)
    db = np.sum(dscores, axis=0, keepdims=True)

    dW += reg * W  # regularization gradient

    # perform a parameter update
    W += -step_size * dW
    b += -step_size * db

print(W)
print(b)


scores = np.dot([0,0,1], W) + b
# compute the class probabilities
exp_scores = np.exp(scores)
probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)  # [N x K]
print(' for [0,0,1] - ', probs)


iteration 0: loss 1.872996
iteration 10: loss 0.243096
iteration 20: loss 0.131005
iteration 30: loss 0.093438
iteration 40: loss 0.075212
iteration 50: loss 0.064682
iteration 60: loss 0.057941
iteration 70: loss 0.053323
iteration 80: loss 0.050001
iteration 90: loss 0.047523
iteration 100: loss 0.045621
iteration 110: loss 0.044126
iteration 120: loss 0.042928
iteration 130: loss 0.041952
iteration 140: loss 0.041146
iteration 150: loss 0.040471
iteration 160: loss 0.039899
iteration 170: loss 0.039411
iteration 180: loss 0.038989
iteration 190: loss 0.038621
[[-3.23369748  5.00542509]
 [ 1.40618408  1.22956322]
 [ 1.91497094 -1.18614417]]
[[ 0.41505245 -0.41505245]]
 for [0,0,1] -  [[0.9807578 0.0192422]]


In [18]:
# nn with 1 hidden layer
#initialize parameters randomly
h = 100 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,C)
b2 = np.zeros((1,C))

# some hyperparameters
step_size = 1e-0
reg = 1e-3 # regularization strength

# gradient descent loop
num_examples = training_set_inputs.shape[0]
for i in range(2000):
  
  # evaluate class scores, [N x C]
  hidden_layer = np.maximum(0, np.dot(training_set_inputs, W) + b) # note, ReLU activation
  scores = np.dot(hidden_layer, W2) + b2
  
  # compute the class probabilities
  exp_scores = np.exp(scores)
  probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x C]
  
  # compute the loss: average cross-entropy loss and regularization
  corect_logprobs = -np.log(probs[range(num_examples),training_set_outputs])
  data_loss = np.sum(corect_logprobs)/num_examples
  reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
  loss = data_loss + reg_loss
  if i % 100 == 0:
    print ("iteration %d: loss %f" % (i, loss))
  
  # compute the gradient on scores
  dscores = probs
  dscores[range(num_examples),training_set_outputs] -= 1
  dscores /= num_examples
  
  # backpropate the gradient to the parameters
  # first backprop into parameters W2 and b2
  dW2 = np.dot(hidden_layer.T, dscores)
  db2 = np.sum(dscores, axis=0, keepdims=True)
  # next backprop into hidden layer
  dhidden = np.dot(dscores, W2.T)
  # backprop the ReLU non-linearity
  dhidden[hidden_layer <= 0] = 0
  # finally into W,b
  dW = np.dot(training_set_inputs.T, dhidden)
  db = np.sum(dhidden, axis=0, keepdims=True)
  
  # add regularization gradient contribution
  dW2 += reg * W2
  dW += reg * W
  
  # perform a parameter update
  W += -step_size * dW
  b += -step_size * db
  W2 += -step_size * dW2
  b2 += -step_size * db2

iteration 0: loss 0.693008
iteration 100: loss 0.011667
iteration 200: loss 0.011240
iteration 300: loss 0.011153
iteration 400: loss 0.011096
iteration 500: loss 0.011049
iteration 600: loss 0.011008
iteration 700: loss 0.010973
iteration 800: loss 0.010941
iteration 900: loss 0.010914
iteration 1000: loss 0.010889
iteration 1100: loss 0.010868
iteration 1200: loss 0.010848
iteration 1300: loss 0.010831
iteration 1400: loss 0.010816
iteration 1500: loss 0.010803
iteration 1600: loss 0.010791
iteration 1700: loss 0.010780
iteration 1800: loss 0.010771
iteration 1900: loss 0.010762


In [20]:
# evaluate training set accuracy
hidden_layer = np.maximum(0, np.dot(training_set_inputs, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis=1)
print ('training accuracy: %.2f' % (np.mean(predicted_class == training_set_outputs)))

training accuracy: 1.00


In [21]:
predicted_class

array([0, 1, 1, 0])

In [24]:
scores =np.dot(np.dot([0,0,1], W)+b,W2)+b2
# compute the class probabilities
exp_scores = np.exp(scores)
probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)  # [N x K]
print(' for [0,0,1] - ', probs)



 for [0,0,1] -  [[0.99843542 0.00156458]]
